In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from forex_python.converter import CurrencyRates

class Portfolio:
    def __init__(self):
        self.assets = []
   
    def add_asset(self):
        # Prompt the user for the asset details
        ticker = input("Enter stock ticker (e.g., AAPL, MSFT): ").upper()
        sector = input("Enter sector (e.g., Tech, Healthcare): ").upper()
        asset_class = input("Enter asset class (e.g., Equity, Bond): ").upper()
        quantity = int(input("Enter quantity of shares: "))
        purchase_price = float(input("Enter purchase price per share: "))
        market = input("Enter market (EAM, TDG, NDQ, XET): ").upper()

        # Calculate transaction cost based on market
        if market == "EAM":
            transaction_cost = 3 
            currency_symbol = "€"
        elif market == "TDG":
            transaction_cost = 3.90 
            currency_symbol = "€"
        elif market == "NDQ":
            transaction_cost = 0.50 
            # Add currency conversion cost (0.25% of the total transaction cost)
            transaction_cost += 0.25 * quantity
            currency_symbol = "$"
        elif market == "XET":
            transaction_cost = 4 
            currency_symbol = "€"
        else:
            print("Invalid market. Please choose from EAM, TDG, NDQ, or XET.")
            return

        stock = yf.Ticker(ticker)
        asset_name = stock.info.get("longName", ticker)
        # Get the real-time price safely
        current_price = stock.info.get("regularMarketPrice")  

        # If the real-time price is missing or None, fallback to the last closing price
        if current_price is None:
            current_price = stock.history(period="1d")["Close"].iloc[-1]
        
        if market == "EAM" or market == "TDG" or market == "XET":
            # Get the exchange rate from USD to EUR
            exchange_rate = self.get_exchange_rate()
            # Convert from USD to EUR
            current_price = current_price * exchange_rate
            current_price = f"{current_price:.2f}"
            
        # Calculate initial profit for the new asset (if it's being added)
        initial_profit = ((float(current_price) - purchase_price) * quantity) - transaction_cost
        initial_profit = f"{initial_profit:.2f}"
    
        # Create a dictionary with the asset data
        asset_data = {
            "asset_name": asset_name,
            "ticker": ticker,
            "sector": sector,
            "asset_class": asset_class,
            "quantity": quantity,
            "purchase_price": purchase_price,
            "market": market,
            "transaction_cost": transaction_cost,
            "currency_symbol": currency_symbol,
            "current_price": current_price,
            "total_profit": initial_profit
        }

        # Check if the asset already exists in the portfolio
        existing_asset = next((asset for asset in self.assets if asset['ticker'] == ticker and asset['asset_class'] == asset_class and asset['market'] == market), None)

        if existing_asset:
            # Calculate old value and new value
            old_value = existing_asset['purchase_price'] * existing_asset['quantity']
            new_value = purchase_price * quantity 

            # Total value (old + new)
            total_value = old_value + new_value

            # New total quantity
            new_quantity_total = existing_asset['quantity'] + quantity

            # Calculate new average purchase price
            average_price = total_value / new_quantity_total

            # Update the asset with new quantity and average price
            existing_asset['quantity'] = new_quantity_total
            existing_asset['purchase_price'] = f"{average_price:.2f}"
            
            # Calculate and add the transaction cost for the current transaction
            existing_asset['transaction_cost'] += transaction_cost
            
            # Calculate the profit for the new quantity based on the new purchase price
            initial_profit = ((float(current_price) - float(existing_asset['purchase_price'])) * existing_asset['quantity']) - existing_asset['transaction_cost']
            
            existing_asset['total_profit'] = f"{initial_profit:.2f}"

        else:
            # Add new asset to portfolio
            self.assets.append(asset_data)

        print(f"{ticker} has been added to your portfolio!")
        
    def get_exchange_rate(self):
        try:
            # First attempt to get exchange rate from forex-python
            cr = CurrencyRates()
            exchange_rate = cr.get_rate('USD', 'EUR')
            return exchange_rate
        except Exception as e:
            # If forex-python fails, use yfinance as fallback
            currency_pair = "USDEUR=X"  # USD to EUR
            data = yf.download(currency_pair, period="1d", interval="1m")
            if not data.empty:
                # Get the most recent exchange rate
                exchange_rate = float(data['Close'].iloc[-1])
                return exchange_rate

    def show_portfolio(self):
        if not self.assets:
            print("Your portfolio is empty.")
            return

        print("\nYour Portfolio:")
        for asset in self.assets:
            print(f"{asset['asset_name']} - {asset['ticker']} - {asset['sector']} ({asset['asset_class']}) - "
                  f"{asset['quantity']} {asset['asset_class']} purchased for an average price of {asset['currency_symbol']}{asset['purchase_price']} - "
                  f"Transaction Cost: €{asset['transaction_cost']} - Current price: {asset['currency_symbol']}{asset['current_price']} - Total profit: {asset['currency_symbol']}{asset['total_profit']}" )

        
    def plot_stock_history(self):
        tickers = input("Enter stock tickers separated by commas (e.g., AAPL, MSFT): ").upper().split(",")
        start_date = start_date = input("Enter start date (YYYY-MM-DD): ")
        end_date = datetime.today().strftime('%Y-%m-%d')

        plt.figure(figsize=(10, 5))

        for ticker in tickers:
            ticker = ticker.strip()
            try:
                stock_data = yf.download(ticker, start=start_date, end=end_date)
                days = range(1, len(stock_data) + 1)  # Use a counter instead of dates
            
                plt.plot(days, stock_data["Close"], label=ticker)
            except Exception as e:
                print(f"Error retrieving data for {ticker}: {e}")

        plt.xlabel("Days Since Start Date")
        plt.ylabel("Closing Price")
        plt.title("Stock Price History")
        plt.legend()
        plt.grid(axis="y", linestyle="--")
        plt.xlim(1, len(stock_data))
        plt.show()
    
    def get_total_portfolio_value(self):
        total_value = 0
        asset_values = {}
        
        # Calculate total portfolio value and individual asset values
        for asset in self.assets:
            current_value = float(asset['current_price']) * asset['quantity']  # Asset value = current price * quantity
            total_value += current_value
            asset_values[asset['asset_name']] = current_value
        
        return total_value, asset_values

    def get_value_by_asset_class(self):
        asset_class_values = {}
        
        # Calculate portfolio value by asset class
        for asset in self.assets:
            current_value = float(asset['current_price']) * asset['quantity']
            if asset['asset_class'] in asset_class_values:
                asset_class_values[asset['asset_class']] += current_value
            else:
                asset_class_values[asset['asset_class']] = current_value
        
        return asset_class_values

    def get_value_by_sector(self):
        sector_values = {}
        
        # Calculate portfolio value by sector
        for asset in self.assets:
            current_value = float(asset['current_price']) * asset['quantity']
            if asset['sector'] in sector_values:
                sector_values[asset['sector']] += current_value
            else:
                sector_values[asset['sector']] = current_value
        
        return sector_values

    def display_values_with_weights(self, total_value, values_dict, label):
        print(f"\n{label}:")
        for name, value in values_dict.items():
            weight = (value / total_value) * 100  # Calculate weight as a percentage
            print(f"{name}: {value:.2f} ({weight:.2f}%)")

    def show_total_value(self):
        total_value, _ = self.get_total_portfolio_value()
        print(f"\nTotal Portfolio Value: {total_value:.2f}")

    def show_values(self):
        # Ask user to choose how they want to view the weights
        print("\nChoose the type of value and weights you want to see:")
        print("1. Value and Weights by Asset")
        print("2. Value and Weights by Asset Class")
        print("3. Value and Weights by Sector")

        value_choice = input("Enter your choice (1/2/3): ")

        total_value, _ = self.get_total_portfolio_value()

        if value_choice == "1":
            # Get value by individual asset
            asset_values = {asset['asset_name']: float(asset['current_price']) * asset['quantity'] for asset in self.assets}
            self.display_values_with_weights(total_value, asset_values, "Value and Weights by Asset")
        elif value_choice == "2":
            # Get value by asset class
            asset_class_values = self.get_value_by_asset_class()
            self.display_values_with_weights(total_value, asset_class_values, "Value and Weights by Asset Class")
        elif value_choice == "3":
            # Get value by sector
            sector_values = self.get_value_by_sector()
            self.display_values_with_weights(total_value, sector_values, "Value and Weights by Sector")
        else:
            print("Invalid choice, please try again.")
          
    def run(self):
        while True:
            # Display the main menu to the user
            print("\nInvestment Portfolio CLI")
            print("1. Add Asset")
            print("2. View Portfolio")
            print("3. Show Historical Prices (Graph)")
            print("4. Calculate Portfolio Value and Weights")
            print("5. Exit")

            choice = input("Choose an option (1/2/3/4/5): ")

            if choice == "1":
                self.add_asset()
            elif choice == "2":
                self.show_portfolio()
            elif choice == "3":
                self.plot_stock_history()
            elif choice == "4":
                self.show_total_value()  # Show the total portfolio value
                self.show_values()  # Show values and weights
            elif choice == "5":
                print("Exiting portfolio tracker. Goodbye!")
                break
            else:
                print("Invalid choice, please try again.")

# run the code
if __name__ == "__main__":
    portfolio = Portfolio()
    portfolio.run()


Investment Portfolio CLI
1. Add Asset
2. View Portfolio
3. Show Historical Prices (Graph)
4. Calculate Portfolio Value and Weights
5. Exit
